In [2]:
import numpy as np
import cv2 #opencv
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

## Setting Training Data

In [3]:
#storing paths to all csv files
import os
csv_name_list=[]

for entry in os.scandir("./trainingData"):
    csv_name_list.append(entry.path)
csv_name_list

['./trainingData\\A.csv',
 './trainingData\\B.csv',
 './trainingData\\C.csv',
 './trainingData\\D.csv']

In [4]:
#giving nos. to each letter/no.

class_dict = {}
count = 0
for i in range(10):
    class_dict[str(i)]=i
    
for i in range(65,91):
    class_dict[chr(i)] = i
class_dict

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 'A': 65,
 'B': 66,
 'C': 67,
 'D': 68,
 'E': 69,
 'F': 70,
 'G': 71,
 'H': 72,
 'I': 73,
 'J': 74,
 'K': 75,
 'L': 76,
 'M': 77,
 'N': 78,
 'O': 79,
 'P': 80,
 'Q': 81,
 'R': 82,
 'S': 83,
 'T': 84,
 'U': 85,
 'V': 86,
 'W': 87,
 'X': 88,
 'Y': 89,
 'Z': 90}

## Stacking different form of img, and creating X&Y

In [7]:

import csvLibrary as cl
X=[]
y=[]

for file in csv_name_list:        
    path = file
    # print(path[15])
    csvData = cl.dread(path)

    for rowEntry in csvData:
        # print(rowEntry)
        tempRow = []
        for handPoint, coords in rowEntry.items():
            if coords != '':
                coords = list(coords[1:-1].split(","))
                # print(float(handPoint), int(coords[0]),int(coords[1]))
                tempRow.extend([float(handPoint), int(coords[0]),int(coords[1])])
            else:
                tempRow.extend([float(handPoint),-600,-600])
        X.append(tempRow)
        y.append(path[15])
# print(X)
# print(y,len(y))

## Time to train our model

In [8]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import pandas as pd

In [10]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC(kernel = 'rbf', C = 10))])
# pipe.fit(X_train, y_train)
# pipe.score(X_test, y_test)

pipe = make_pipeline(StandardScaler(), svm.SVC(gamma='auto',probability=False))
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.5769230769230769

In [12]:
print(classification_report(y_test, pipe.predict(X_test),zero_division=np.nan))

              precision    recall  f1-score   support

           A       0.50      0.15      0.24        13
           B       0.27      0.23      0.25        13
           C       0.94      1.00      0.97        17
           D       0.42      0.89      0.57         9

    accuracy                           0.58        52
   macro avg       0.53      0.57      0.51        52
weighted avg       0.57      0.58      0.54        52



## Save Model & Class Dictionary

In [15]:
# !pip install joblib
import joblib 
# Save the model as a pickle in a file 
joblib.dump(pipe, './artifacts/saved_model.pkl') 


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


['saved_model.pkl']

In [14]:
import json
with open("./artifacts/class_dictionary.json","w") as f:
    f.write(json.dumps(class_dict))